# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = get_date_list(data)
data = read_data(data)

Read dataframe with 16797607 records.
Dataframe has 71828 records between (datetime.datetime(2021, 2, 3, 0, 0), datetime.datetime(2021, 2, 3, 0, 0)).
Subsetted dataframe with 71828 records.
Record num : 71828
Wall time: 17.3 s


In [3]:
for data['i'] in tqdm(range(0, data['df_len'])):
    
    data['df']['DateTime_frmt'][data['i']] = dt.datetime.strptime(data['df']['DateTime'][data['i']].split(".")[0],"%Y%m%d %H:%M:%S")
    
    data['ask'] = data['df']['Ask'][data['i']]
    data['bid'] = data['df']['Bid'][data['i']]  
    
    data['tick'] = (data['df']['Ask'][data['i']] + data['df']['Bid'][data['i']])/2
    data['df']['tick'][data['i']] = data['tick']
        
    # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------
    
        # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------


#     # lema before after loops --------------------------------
#     if len(data['lema_list']) < data['lema_len']:
#         data = before_lema(data)        
#         continue

#     if len(data['lema_list']) == data['lema_len']:
#         data = after_lema(data)                  
#     # ----------------------------------------------------------
    
    data = get_ohlc(data)    
    
#     data = get_position(data)
#     if not data['position']:
#         continue
        
#     # Get Dirs --------------------------------
#     if len(data['dir_list']) < 2:
#         data['dir_list'].append(data['position'])   
#         continue

#     elif len(data['dir_list']) == 2:
#         data = get_cross_dir(data)
#     # ----------------------------------------------------------  
    
    data = get_trend_fwd(data)
    data = trend_angle_check(data)

100%|███████████████████████████████████| 71828/71828 [08:00<00:00, 149.44it/s]


In [4]:
data['df']

,DateTime,Bid,Ask,tick,DateTime_frmt,sema,lema,position,direction,to_order,h_line_angle,l_line_angle,trend_angle,h_trend_calc_spot,l_trend_calc_spot,h,l
0,20210203 00:00:00.267,1.20365,1.20371,1.203680,2021-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210203 00:00:00.419,1.20366,1.20371,1.203685,2021-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210203 00:00:00.723,1.20368,1.20370,1.203690,2021-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210203 00:00:00.979,1.20366,1.20371,1.203685,2021-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210203 00:00:01.182,1.20367,1.20371,1.203690,2021-02-03 00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71823,20210203 23:59:48.931,1.20411,1.20416,1.204135,2021-02-03 23:59:48,1.204039,NaN,NaN,NaN,NaN,5.992309,4.59242,NaN,1.203849,1.203009,NaN,NaN
71824,20210203 23:59:50.853,1.20412,1.20416,1.204140,2021-02-03 23:59:50,1.204040,NaN,NaN,NaN,NaN,5.992309,4.59242,NaN,1.203849,1.203009,NaN,NaN
71825,20210203 23:59:53.016,1.20413,1.20416,1.204145,2021-02-03 23:59:53,1.204042,NaN,NaN,NaN,NaN,5.992309,4.59242,NaN,1.203849,1.203009,NaN,NaN
71826,20210203 23:59:53.118,1.20413,1.20417,1.204150,2021-02-03 23:59:53,1.204044,NaN,NaN,NaN,NaN,5.992309,4.59242,NaN,1.203849,1.203010,NaN,NaN


In [5]:
num_rec = len(data["df"])
hours = np.round((list(data['df']['DateTime_frmt'])[-1] - list(data['df']['DateTime_frmt'])[0]).total_seconds() / 60 / 60 , 2)
recsphour = int(num_rec/hours)

print(f'Records   : {num_rec}')
print(f'Hours     : {hours}')
print(f'Recs/hour : {recsphour}')
print(f'Recs/min  : {int(recsphour/60)}')
print(f'Recs/sec  : {np.round(recsphour/3600, 1)}')

data['df'].head()

Records   : 71828
Hours     : 24.0
Recs/hour : 2992
Recs/min  : 49
Recs/sec  : 0.8


,DateTime,Bid,Ask,tick,DateTime_frmt,sema,lema,position,direction,to_order,h_line_angle,l_line_angle,trend_angle,h_trend_calc_spot,l_trend_calc_spot,h,l
0,20210203 00:00:00.267,1.20365,1.20371,1.203680,2021-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210203 00:00:00.419,1.20366,1.20371,1.203685,2021-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210203 00:00:00.723,1.20368,1.20370,1.203690,2021-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210203 00:00:00.979,1.20366,1.20371,1.203685,2021-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210203 00:00:01.182,1.20367,1.20371,1.203690,2021-02-03 00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
plot_graph(data)

In [7]:
data['df'].to_csv('data/temp.csv')

In [8]:
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

In [9]:
send_telegram_message(f'Run Complete at : {dt.datetime.now()}')